## 교차검증

- 학습용 데이터와 테스트용 데이터를 분리해서 모데링 및 평가를 수행하는 방식은 과적합에 취약한 약점을 가질 수 있다.
- 과적합은 모델이 학습 데이터에만 과도하게 최적화되어 실제 예측을 다른 데이터로 수행할 떄 예측 성능이 과도하게 떨어 지는 것을 의미한다.
- 고정된 테스트 데이터로만 평가를 하게되면 테스트 데이터에만 최적의 성능을 발휘할 수 있도록 편향되게 모델을 유도하는 경향이 발생한다.
- 이러한 문제점을 개선하기 위하여 교차 검증을 이용해 다양한 학습과 평가를 수행한다.


## K 폴드

- K개의 데이터 폴드 세트를 만들어서 K번 만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행하는 방법
- 데이터 세트를 K등분 , 학습 데이터와 검증 데이터 세트를 K 번 변경하면서 학습과 검증을 수행한 결과를 평균해서 평과 결과 산출
- 사이킷런의 KFold와 StratifiledKFold 클래스를 제공

In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.datasets import load_iris

iris = load_iris()
iris.keys()

dt_clf = DecisionTreeClassifier()

train_data = iris.data
train_label = iris.target

dt_clf.fit(train_data, train_label)

pred = dt_clf.predict(train_data)

accuracy_score(train_label, pred)

1.0

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target,
                                                   test_size=0.3,
                                                   random_state=1)

dt_clf.fit(X_train,y_train)
split_predict = dt_clf.predict(X_test)
accuracy_score(y_test,split_predict)

0.9555555555555556

In [20]:
import numpy as np

features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성

kfold = KFold(n_splits = 5)
cv_accuracy = []

n_iter = 0

for train_index,test_index in kfold.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    
    accuracy = np.round(accuracy_score(y_test, pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차검증 정확도 : {1}, 학습데이터크기 : {2}, 검증 데이터 크기: {3}'
         .format(n_iter, accuracy , train_size, test_size))
    cv_accuracy.append(accuracy)
    
print('\n## 평균 검증 정확도: ',np.mean(cv_accuracy))


#1 교차검증 정확도 : 1.0, 학습데이터크기 : 120, 검증 데이터 크기: 30

#2 교차검증 정확도 : 0.9667, 학습데이터크기 : 120, 검증 데이터 크기: 30

#3 교차검증 정확도 : 0.8667, 학습데이터크기 : 120, 검증 데이터 크기: 30

#4 교차검증 정확도 : 0.9333, 학습데이터크기 : 120, 검증 데이터 크기: 30

#5 교차검증 정확도 : 0.7333, 학습데이터크기 : 120, 검증 데이터 크기: 30

## 평균 검증 정확도:  0.9


### Stratified K폴드

- 불균형한 분포도를 가진 레이블 데이터 집합을 위한 K폴드 방식
- 불균형한 분포도를 가진 레이블 데이터 집합은 특정 레이블 값이 특이하게 많거나 매우 적어서 값의 분포가 한쪽으로 치우치는 것을 의미한다.
  대출 사기 데이터를 예측하는 경우 대부분의 데이터가 정상 데이터이므로 대출 사기 레이블의 비율이 아주 적게 될 수 있으며 레이블 값 1이 특정
  개별 반복별 학습/테스트 데이터 세트에는 상대적으로 많이 들어가고 다른 데이터 세트에는 그렇지 않을 수가 있다.
- 대출 사기 레이블이 1인 레코드는 비록 건수는 작지만 알고리즘이 대출 사기를 예측하기 위한 중요한 파처를 가지고 있으므로 중요한 데이터 세트로서
  원본 데이터와 유사한 대출 사기 레이블 값의 분포를 학습/테스트 세트에도 유지하는게 중요하다.
- Stratified KFold는 KFold가 레이블 데이터 집합이 원본 데이터 집합의 레이블 분포를 학습 및 테스트 세트에 제대로 분배 하지 못하는 경우의 문제를 해결해준다.
- 일반적으로 분류에서의 교차검증은 Stratified KFold로 분할되어야 한다.
- 회귀의 결정값은 이산값 형태의 레이블이 아니라 연속된 숫자이기 떄문에 결정값별로 분포를 정하는 것이 의미가 없으므로 Stratified KFold가 지원되지 않는다.

In [21]:
import pandas as pd
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df.label.value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [29]:
kfold = KFold(n_splits=3)
n_iter = 0

for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    
    print('##교차검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n',label_train.value_counts())
    print('검증 레이블 데이터 분포:\n',label_test.value_counts())

##교차검증 : 1
학습 레이블 데이터 분포:
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
##교차검증 : 2
학습 레이블 데이터 분포:
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
##교차검증 : 3
학습 레이블 데이터 분포:
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


In [32]:
from sklearn.model_selection import StratifiedKFold

skt = StratifiedKFold(n_splits=3)
n_iter = 0

for train_index, test_index in skt.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train  = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증:{0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n',label_train.value_counts())
    print('검증 레이블 데이터 분포:\n',label_test.value_counts())

## 교차 검증:1
학습 레이블 데이터 분포:
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
1    17
2    16
Name: label, dtype: int64
## 교차 검증:2
학습 레이블 데이터 분포:
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
2    17
1    16
Name: label, dtype: int64
## 교차 검증:3
학습 레이블 데이터 분포:
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
2    17
0    16
Name: label, dtype: int64


In [ ]:
# 과제
# KFold 대신 StratifiedKFold를 적용하여 상기건을 평가하세요



### cross_val_score()

- 사이킷런이 제공하는 교차검증을 좀 더 편리하게 수행할 수있는 대표적인 API 이다
- KFold의 일련의 과정을 한꺼번에 수행해준다.
- 주요 파라미터는 estimator,X,y,scroing,cv이다.
- cross_val_score()는 내부적으로 StratifiedKFold를 이용한다
- cross_val_score()는 하나의 평가 지표만 가능하나 corss_validate()는 여러 개의 평가 지표를 반환한다.

In [39]:
from sklearn.model_selection import cross_val_score, cross_validate

data = iris.data
label = iris.target

scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print('교차 검증별 정확도: ',np.round(scores,4))
print('평균: ',np.round(np.mean(scores),4))

교차 검증별 정확도:  [0.98 0.92 1.  ]
평균:  0.9667


## GridSearchCV

- 하이퍼 파라미터는 ML알고리즘의 주요 구성 요소이며 이 값을 조정해 알고리즘의 예측 성능을 개선할 수 있다.
- 사이킷런은 GridSearchCV API를 이용해 Classifier나 Regressor와 같은 알고리즘에 사용되는 하이퍼 파라미터를 순차적으로 입력하면서
  편리하게 최적의 파라미터를 도출 할 수있는 방안을 제공한다
- GridSearchCV 는 교차 검증을 기반으로 이 하이퍼파라미터의 최적 값을 찾게 해준다. 즉 데이터 세트를 cross_validation을 위한
  학습/테스트 세트로 자동분할한 뒤에 하이퍼 파라미터 그리드에 기술된 모든 파라미터를 순차적으로 적용해 최적의 파라미터를 찾을 수 있게 해준다.
- CV가 3회, 6개의 파라미터 조합이라면 총 18회의 학습/평가가 이루어진다.
- 주요 파라미터 : estimator , param_grid, scroing, cv, refit
    - param_grid : estimator 튜닝을 위해 파라미터명과 사용될 여러 파라미터값을 지정
    - refit : 디폴트가 True이며 True로 생성 시 가장 최적의 파라미터를 찾은 뒤 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습시킨다.

In [9]:
# 과제
# Q. iris 데이터셋을 GridSearchCV를 이용하여 수행하세요
# - randomforest, logisticRegression, svm

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


iris = load_iris()
iris.keys()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target,
                                                   test_size=0.3, random_state=121) 

rfc = RandomForestClassifier()
lr = LogisticRegression()
svc = SVC()

models = [rfc,lr,svc]

for model in models:
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    print(model, accuracy_score(y_test,pred))
    

RandomForestClassifier() 0.9333333333333333
LogisticRegression() 0.9555555555555556
SVC() 0.9555555555555556


In [16]:
svc_params = svc.get_params()

In [17]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score

grid_rfc = GridSearchCV(svc,param_grid = svc_params)
grid_rfc

ValueError: Parameter grid for parameter (C) needs to be a list or numpy array, but got (<class 'float'>). Single values need to be wrapped in a list with one element.